In [4]:
import numpy as np
import sys

def generate_spiral_data(N, K):
    X = np.zeros((N*K, 2))
    y_raw = np.zeros(N*K, dtype='uint8')
    y = np.zeros((N*K, K))
    
    for k in range(K):
        ix = range(N*k, N*(k+1))
        r = np.linspace(0.0, 1, N)
        t = np.linspace(k*4, (k+1)*4, N) + np.random.randn(N)*0.2
        X[ix] = np.c_[r*np.sin(t*2.5), r*np.cos(t*2.5)]
        y_raw[ix] = k
        y[ix, k] = 1
        
    return X, y, y_raw

class NeuralNetwork(object):
    def __init__(self, input_nodes, hidden1_nodes, hidden2_nodes, output_nodes, learning_rate):
        self.input_nodes = input_nodes
        self.hidden1_nodes = hidden1_nodes
        self.hidden2_nodes = hidden2_nodes
        self.output_nodes = output_nodes
        self.lr = learning_rate
        
        self.W_IH1 = np.random.randn(self.input_nodes, self.hidden1_nodes) * 0.01
        self.b1 = np.zeros((1, self.hidden1_nodes))
        
        self.W_H1H2 = np.random.randn(self.hidden1_nodes, self.hidden2_nodes) * 0.01
        self.b2 = np.zeros((1, self.hidden2_nodes))
        
        self.W_H2O = np.random.randn(self.hidden2_nodes, self.output_nodes) * 0.01
        self.b_out = np.zeros((1, self.output_nodes))

    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        return (x > 0) * 1

    def softmax(self, x):
        exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_values / np.sum(exp_values, axis=1, keepdims=True)

    def feedForward(self, X):
        self.h1_input = np.dot(X, self.W_IH1) + self.b1
        self.h1_output = self.relu(self.h1_input)
        
        self.h2_input = np.dot(self.h1_output, self.W_H1H2) + self.b2
        self.h2_output = self.relu(self.h2_input)
        
        output_input = np.dot(self.h2_output, self.W_H2O) + self.b_out
        pred = self.softmax(output_input)
        
        return pred
        
    def backPropagation(self, X, Y, pred):
        output_delta = Y - pred
        
        h2_error = output_delta.dot(self.W_H2O.T)
        h2_delta = h2_error * self.relu_derivative(self.h2_output)
        
        h1_error = h2_delta.dot(self.W_H1H2.T)
        h1_delta = h1_error * self.relu_derivative(self.h1_output)
        
        self.W_H2O += self.h2_output.T.dot(output_delta) * self.lr
        self.b_out += np.sum(output_delta, axis=0, keepdims=True) * self.lr
        
        self.W_H1H2 += self.h1_output.T.dot(h2_delta) * self.lr
        self.b2 += np.sum(h2_delta, axis=0, keepdims=True) * self.lr
        
        self.W_IH1 += X.T.dot(h1_delta) * self.lr
        self.b1 += np.sum(h1_delta, axis=0, keepdims=True) * self.lr
        
    def train(self, X, Y):
        pred = self.feedForward(X)
        self.backPropagation(X, Y, pred)
        
    def calculate_loss(self, X, Y):
        num_examples = len(X)
        pred = self.feedForward(X)
        corect_logprobs = -np.log(pred[range(num_examples), np.argmax(Y, axis=1)] + 1e-9)
        loss = np.sum(corect_logprobs)
        return loss / num_examples

    def predict(self, X):
        pred = self.feedForward(X)
        return np.argmax(pred, axis=1)

print("Generating Data ")
N = 167
K = 3
X, y, y_raw = generate_spiral_data(N, K)
print(f"Generated {len(X)} data points.")

print("\n Initializing 3-Layer NN ")
input_size = 2
h1_size = 10
h2_size = 10
output_size = 3
learning_rate = 0.1

nn = NeuralNetwork(input_nodes=input_size, 
                   hidden1_nodes=h1_size, 
                   hidden2_nodes=h2_size, 
                   output_nodes=output_size, 
                   learning_rate=learning_rate)
                   
epochs = 10000
losses = []

for i in range(epochs + 1):
    nn.train(X, y)
    
    if i % 1000 == 0:
        loss = nn.calculate_loss(X, y)
        sys.stdout.write(f'Epoch {i}, Loss: {loss:.4f}\n')



print("\n Accuracy ")
predictions = nn.predict(X)
actual = np.argmax(y, axis=1)
accuracy = np.mean(predictions == actual)
print(f'Accuracy: {accuracy}')

Generating Data 
Generated 501 data points.

 Initializing 3-Layer NN 
Epoch 0, Loss: 1.0986
Epoch 1000, Loss: 9.9300
Epoch 2000, Loss: 10.4400
Epoch 3000, Loss: 13.8153
Epoch 4000, Loss: 10.5876
Epoch 5000, Loss: 10.7115
Epoch 6000, Loss: 13.8148
Epoch 7000, Loss: 10.7955
Epoch 8000, Loss: 10.8587
Epoch 9000, Loss: 13.8141
Epoch 10000, Loss: 10.9190

 Accuracy 
Accuracy: 0.3333333333333333
